In [14]:
from tools.preprocess import *
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import KFold, cross_val_score


pd.options.display.max_rows=140
pd.options.display.max_columns=140

In [15]:
X_train_0, Y_train_0, X_test_0, X_test_ids = load_data('data/', False)

In [16]:
preprocessing_parameters = {
    'drop_columns':["exposition","energy_performance_category","ghg_category"],
    'frequency_encoding':['city'],
    'quantile_encoding':[],
    'knn_imputation':["size", "land_size","energy_performance_value","ghg_value", "nb_rooms","nb_bathrooms", "nb_bedrooms"],
    'feature_engineering':False,
    'add_geo':True,
    'feature_transformation':False,
    'hot_encoding':True,
    'target_transformation':True,
}

In [17]:
X_train_1,Y_train_1, X_test_1 = preprocess(X_train_0, Y_train_0, X_test_0,preprocessing_parameters)

In [7]:
#baseline model
baseline_model=XGBRegressor()
baseline_model.fit(X_train_1, Y_train_1)
kf = KFold(n_splits=10)
baseline_result = cross_val_score(baseline_model, X_train_1, Y_train_1, scoring="neg_mean_absolute_error", cv=kf)
mean_baseline_result = -np.mean(baseline_result)
mean_baseline_result

0.28905175802891553

In [18]:
xgb_params={'tree_method': 'auto',
 'learning_rate': 0.22502761577232197,
 'max_depth': 7,
 'min_child_weight': 2.8420515790248526,
 'subsample': 0.975,
 'colsample_bytree': 0.75,
 'colsample_bylevel': 0.925,
 'gamma': 0.001531824642793309,
 'reg_lambda': 0.010527402680638263,
 'reg_alpha': 1.2647545029756599e-08}

etr_params={
}

cat_params={
    'learning_rate': 0.17447854966066342,
    'objective': 'RMSE',
    'colsample_bylevel': 0.09999300796304286,
    'depth': 12,
    'boosting_type': 'Plain',
    'bootstrap_type': 'Bernoulli',
    'subsample': 0.9709524520446949
}

In [19]:
tuned_models = {
    "xgboost":XGBRegressor(**xgb_params),
    #"rfr":RandomForestRegressor(),
    "etr":ExtraTreesRegressor(),
    "cat":CatBoostRegressor(**cat_params, verbose=False),
}


for model_name, model in tuned_models.items():
    model.fit(X_train_1, Y_train_1)
    print(f"Finished Training {model_name}.")


Finished Training xgboost.
Finished Training etr.
Finished Training cat.


In [24]:
final_predictions_sum= (
    7/20*np.exp(tuned_models["xgboost"].predict(X_test_1)) +
    4/20*np.exp(tuned_models["etr"].predict(X_test_1)) + 
    9/20*np.exp(tuned_models["cat"].predict(X_test_1)) 
    )

final_predictions=pd.Series(final_predictions_sum,name="price")
final_predictions.head()
final_submission = pd.concat([X_test_ids, final_predictions], axis=1)
final_submission.to_csv("data/final_submission_52.csv", index=False, header=True)
print("Finished submitting")

Finished submitting
